### Etl Tip


In [8]:

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
import os
import json
import ast
import gzip
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
%matplotlib inline
import nltk
from nltk.corpus import stopwords
import wordcloud
from wordcloud import WordCloud, STOPWORDS
from datetime import datetime

In [10]:

# Definir la ruta del archivo Parquet
ruta_archivo = 'c:\\Users\\Sofia\\Desktop\\tip.parquet'

# Leer el archivo Parquet y cargarlo en un DataFrame
df_tip = pd.read_parquet(ruta_archivo)

# Mostrar las primeras filas para verificar la carga
print(df_tip.head())

                  user_id             business_id  \
0  AGNUgVwnZUey3gcPCJ76iw  3uLgwr0qeCNMjKenHJwPGQ   
1  NBN4MgHP9D3cw--SnauTkA  QoezRbYQncpRqyrLH6Iqjg   
2  -copOvldyKh1qr-vzkDEvw  MYoRNLb5chwjQe3c_k37Gg   
3  FjMQVZjSqY8syIO-53KFKw  hV-bABTK-glh5wj31ps_Jw   
4  ld0AperBXk1h6UbqmM80zw  _uN0OudeJ3Zl_tf6nxg5ww   

                                                text                date  \
0                     Avengers time with the ladies. 2012-05-18 02:17:21   
1  They have lots of good deserts and tasty cuban... 2013-02-05 18:35:10   
2             It's open even when you think it isn't 2013-08-18 00:56:08   
3                          Very decent fried chicken 2017-06-27 23:05:38   
4             Appetizers.. platter special for lunch 2012-10-06 19:43:09   

   compliment_count  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  


In [3]:
df_tip.sample(5)

,user_id,business_id,text,date,compliment_count
262169,10yZLcySWgd3i2cTtRGkvg,ZtAO8lm9H05HSaPx47ArVA,One of the few places around that serves Maryl...,2014-08-03 02:04:28,0
93201,ZQi_M3cUzOajZI-pt1lbHQ,qmiz3iYOn4pZylOe27OTUQ,"Terrible service, food was ok.",2013-05-12 23:34:23,0
565654,d_wcOwjrWTaAUXuYzCJAHw,49Gyz-LD9MxTh1yzbYiOhw,Great coffee and friendly staff!,2017-07-09 18:11:28,0
483333,xkh2PF-psmoqSx7HWr_-TQ,c_4c5rJECZSfNgFj7frwHQ,Love this airport clean and easy to navigate a...,2018-07-30 19:30:06,0
479476,YqAUoP_mO9v7MYuxYVXuIQ,q0Q-ohZp8iV9OY-gkNPPwg,Excellent,2019-05-18 18:03:28,0


In [11]:
df_tip.shape

(908915, 5)

In [12]:
df_tip.info

<bound method DataFrame.info of                        user_id             business_id  \
0       AGNUgVwnZUey3gcPCJ76iw  3uLgwr0qeCNMjKenHJwPGQ   
1       NBN4MgHP9D3cw--SnauTkA  QoezRbYQncpRqyrLH6Iqjg   
2       -copOvldyKh1qr-vzkDEvw  MYoRNLb5chwjQe3c_k37Gg   
3       FjMQVZjSqY8syIO-53KFKw  hV-bABTK-glh5wj31ps_Jw   
4       ld0AperBXk1h6UbqmM80zw  _uN0OudeJ3Zl_tf6nxg5ww   
...                        ...                     ...   
908910  eYodOTF8pkqKPzHkcxZs-Q  3lHTewuKFt5IImbXJoFeDQ   
908911  1uxtQAuJ2T5Xwa_wp7kUnA  OaGf0Dp56ARhQwIDT90w_g   
908912  v48Spe6WEpqehsF2xQADpg  hYnMeAO77RGyTtIzUSKYzQ   
908913  ckqKGM2hl7I9Chp5IpAhkw  s2eyoTuJrcP7I_XyjdhUHQ   
908914  4tF1CWdMxvvwpUIgGsDygA  _cb1Vg1NIWry8UA0jyuXnQ   

                                                     text                date  \
0                          Avengers time with the ladies. 2012-05-18 02:17:21   
1       They have lots of good deserts and tasty cuban... 2013-02-05 18:35:10   
2                  It's open

In [13]:
df_tip.isnull().sum()

user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64

In [14]:

df_tip.duplicated().sum()

np.int64(67)

In [15]:
df_gral_tip = df_tip.drop_duplicates()

In [16]:
df_gral_tip.duplicated()


0         False
1         False
2         False
3         False
4         False
          ...  
908910    False
908911    False
908912    False
908913    False
908914    False
Length: 908848, dtype: bool

In [17]:
# Desanidar la columna "date"
desanidados = []  # Lista vacía para almacenar los datos desanidados.

for _, row in df_gral_tip.iterrows():
    user_id = row['user_id']
    business_id = row['business_id']
    text = row['text']

    for datetime_str in str(row['date']).split(', '):
        datetime_parts = datetime_str.split(' ')
        date = datetime_parts[0]
        hour = datetime_parts[1] if len(datetime_parts) > 1 else None  # Usamos None si no hay hora disponible
        complement_count = row['compliment_count']  # Puedes incluir otras columnas según sea necesario
        desanidados.append([user_id, business_id, text, date, hour, complement_count])

# Crear DataFrame desanidado
df_desanidado = pd.DataFrame(desanidados, columns=['user_id', 'business_id', 'text', 'date', 'hour', 'compliment_count'])

# Mostrar el DataFrame desanidado
print(df_desanidado)

                       user_id             business_id  \
0       AGNUgVwnZUey3gcPCJ76iw  3uLgwr0qeCNMjKenHJwPGQ   
1       NBN4MgHP9D3cw--SnauTkA  QoezRbYQncpRqyrLH6Iqjg   
2       -copOvldyKh1qr-vzkDEvw  MYoRNLb5chwjQe3c_k37Gg   
3       FjMQVZjSqY8syIO-53KFKw  hV-bABTK-glh5wj31ps_Jw   
4       ld0AperBXk1h6UbqmM80zw  _uN0OudeJ3Zl_tf6nxg5ww   
...                        ...                     ...   
908843  eYodOTF8pkqKPzHkcxZs-Q  3lHTewuKFt5IImbXJoFeDQ   
908844  1uxtQAuJ2T5Xwa_wp7kUnA  OaGf0Dp56ARhQwIDT90w_g   
908845  v48Spe6WEpqehsF2xQADpg  hYnMeAO77RGyTtIzUSKYzQ   
908846  ckqKGM2hl7I9Chp5IpAhkw  s2eyoTuJrcP7I_XyjdhUHQ   
908847  4tF1CWdMxvvwpUIgGsDygA  _cb1Vg1NIWry8UA0jyuXnQ   

                                                     text        date  \
0                          Avengers time with the ladies.  2012-05-18   
1       They have lots of good deserts and tasty cuban...  2013-02-05   
2                  It's open even when you think it isn't  2013-08-18   
3          

In [18]:
df_desanidado.head(4)

,user_id,business_id,text,date,hour,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18,02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05,18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18,00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27,23:05:38,0


In [19]:
df_desanidado.to_parquet("c:\\Users\\Sofia\\Desktop\\Archivo desanidado\\Tip.parquet", index=True)